In [ ]:
#first import the code and its functions
import alloy_surface_simulator

from alloy_surface_simulator import calcLC, createSurface, cns, findCNS  

from alloy_surface_simulator import readability, readability_new, csv_read, parameterLookup, energyCalculator, energyCalculator_FAST, atomSwapper, PT_step, PT_simulation, pairSiteCalculator, distributionCalculator, cnCalculator 

from alloy_surface_simulator import exportData, runSimulation, readData, equilibrate, boxcar, obtainBaseline

#also import parameter libraries
from alloy_surface_simulator import WF_index, WF_1, WF_2, BCE, BCN, LCN 

The code is equipped to generate a random configuration of a binary alloy nanomaterial, output as a .xyz file specifying the coordinates. You as the user can specify the identities of the metals, the nominal composition of each, and the lattice geometry (Miller index). Skip this step if you already have a configuration in .xyz format.

In [ ]:
e1 = 'Pt' #first element - nominal composition will be expressed in terms of it
e2 = 'Au' #second element
per1 = 50 #percentage of first element in the alloy

lc=calcLC(LCN[e1], LCN[e2], per1) #calculates lattice constant using Vegard's law approximation

dimensions= (5,6,3)

miller_index=100

filename='demo.xyz'

createSurface(dimensions, miller_index, lc, e1, per1, e2, filename) #generates configuration 

The code as written was used for Au-Pt alloys. If you want to use other metals you simply have to append their parameter values. The LCN (lattice constant number) and BCN (bulk coordination number) dictionaries contain no metals other than Pt and Au, so values must be added for all other metals. The weight factors and BCE (bulk cohesive energies) include most metals.

In [ ]:
#verify that Pd is in the BCE dictionary - statement evaluates to True
'Pd' in BCE.keys()

In [ ]:
#add Pd values to LCN and BCN dictionaries

LCN['Pd']=3.89

BCN['Pd']=6

LCN, BCN

In order to properly calculate (and subsequently minimize) the energy of a configuration, we need to know the coordination numbers and nearest neighbors of each atom. Calling `findCNS` will automatically add this info to the .xyz file for our configuration.

In [ ]:
findCNS('demo.xyz')

Now we can determine and store the cohesive energies, bulk coordination numbers, and weight factors for each element in the alloy. We can also calculate the energy of our configuration. 

There is an option to add an adsorbate to the surface of the alloy, or use default of `None` for an adsorbate-free surface (e.g. an alloy in vacuum). Note that the only adsorbate the code is currently equipped to handle is oxygen. For other adsorbates you must append the BCN (always 2 for an adsorbate, because it is on the surface and will only be surrounded by one other atom) and the BCE. You must also modify WF_1 and WF_2 to include the bond dissociation energy for the bond between oxygen and each of the metals in your alloy. WF_1 energies are expressed in eV and WF_2 energies are expressed in kJ/mol; perform a literature search for your desired values.

In [ ]:
#call the readability function so the parameter lookup function can properly read in the file
config = readability('demo.xyz') 

#look up parameters
parameters=parameterLookup(config, WF_1, WF_2, WF_index, BCE, adsorb=None)
atom_types=parameters[0]
BCEs = parameters[1]
BCNs = parameters[2]
weightfactors = parameters[3]

atom_types, BCEs, BCNs, weightfactors

In [ ]:
#calculate energy

total_energy= energyCalculator(config, atom_types,BCEs, BCNs, weightfactors, cover=1)
#here cover refers to percentage of surface that has adsorbates bonded to it. 
#if you have no adsorbates, any number between 0 and 1 is a valid input and won't affect calculations

energy_peratom = total_energy/(5*6*3) #divide by product of the dimensions to get energy per atom

To run a parallel-tempering simulation, two atoms are randomly chosen in the configuration, and their places are swapped. The energies are compared, and if the new configuration has a lower energy than the original OR the new energy is higher but the Boltzmann probability criteria is met, the new configuration is accepted.

In [ ]:
kT = 0.025 #room temperature in eV multiplied by the Boltzmann constant.
#this is used in the probability formula that determines whether a proposed swap is accepted

PT_step(config, kT, atom_types,BCEs, BCNs, weightfactors, cover=1)

Run enough trials, and eventually you'll get tons of configurations in the sample space. Once you discard the initial trials that are required for the system to equilibriate in energy, you can take averages of the energies of all the configurations in the sample space, and this is an accurate portrayal of the energy we would expect our nanomaterial to have "in real life."

Use `PT_simulation()` to loop over the desired number of trials (how many it takes to get an accurate portrayal of the sample space is trial and error). The way the code is structured, 3 different simulations are run at 3 different temperatures, and a fraction (literature suggests 10%, or 0.1) of configurations are swapped between the temperature reservoirs; this prevents the configuration from getting "stuck" in an energy minimum.

Note that it will take a long time and a lot of computational power to run this function, especially if you're using a large number of trials.

In [ ]:
#suggested literature values for reservoir temperatures and fraction of configurations that are swapped
kT_cold=0.025
kT_med=0.035
kT_hot=0.045
frac_swap=0.1

trials=500000 #number of trials

run = PT_simulation('demo.xyz', trials, frac_swap, kTcold, kTmed, kThot, atom_types,BCEs, BCNs,weightfactors, cover=1)

As output, you will get the identities of the elements `run[0]`, the energies of each configuration in each of the temperature reservoirs (for cold reservoir use `run[1]`, pairs of each element on the surface (`run[2]` and `run[3]`), and number of each element on the surface (`run[4]` and `run[5]`).

The `runSimulation` function does all of the above steps, beginning with generating a random configuration, in a single command. It also exports the energy, pair site, and surface distribution data in a .csv file for convenient storage and analysis.

In [ ]:
runSimulation('Pt', 'Au', 50, 5,6,3, 100, 'demo.xyz', None, 500000, 0.025, 0.035, 0.045, cover=1)

How do you know if you've run a good amount of trials, and, if so, how many of the initial trials to discard as part of equilibration? Plot the energy as a function of trial number and note the trends.
*Note: you can also generate these graphs and do analysis in Excel or any other graphing program.*

In [ ]:
#turn the csv file into a readable format
data= readData('demo.csv')

#the energies will be the first list returned by the above function. these are the y-values
y=data[0]
x=[i for i in range(len(y))] #x values are the trial numbers

matplotlib.pyplot.plot(x,y)

As you can see, energy wildly fluctuates until about 10,000 trials and then seems to settle into an equilibrium, where it still changes, but not wildly. That means the 500,000 trials run in this simulation offers an accurate portrayal of the system properties, upon discarding the first 10,000 or so.

Use `equilibriate()` to discard the first 10,000 trials from the data set. You then have a list of the energies from each configuration after the 10,000th, to use for analysis. You can also discard the first 10,000 trials for the other properties, like pair sites.

In [ ]:
keep_energies = equilibriate(run[0], cutoff=10000)

Perhaps you wish to use this code to compare the energies and properties of alloys of the same metals, in which you vary the nominal composition. You may compare alloys with adsorbates to those in vacuum. You might choose to vary the metals and see if one metal tends to prefer being on the surface of the alloy, and if this depends on the other metal it is bonded to (energetically, the higher the bond dissociation energy, the more bonds a species wants to form, and it is these species that gravitate towards higher coordination numbers on the inside of an alloy). You are limited by nothing other than your imagination! (and the computing power of your CPU)